In [3]:
import sys
import datetime as dt
sys.path.insert(0,'../..')
import matplotlib.pyplot as plt
from rivapy.instruments import ZeroCouponBondSpecification, FixedRateBond
from rivapy.instruments.factory import create
from rivapy.tools.enums import DayCounterType, RollConvention, SecuritizationLevel, Currency

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [4]:
import datetime as dt
import pandas as pd
import rivapy.tools.interfaces as interfaces
from rivapy.instruments.factory import create as _create

class InstrumentMemoryStorage:
    def __init__(self):
        self._store = {}
        
    def add(self, instrument: interfaces.FactoryObject):
        if instrument.obj_id in self._store.keys():
            tmp = self._store[instrument.obj_id][-1]
            ins_dict = instrument.to_dict()
            if interfaces.FactoryObject.hash_for_dict(tmp) != interfaces.FactoryObject.hash_for_dict(ins_dict):
                self._store[instrument.obj_id].append(ins_dict)
        else:
            self._store[instrument.obj_id] = [instrument.to_dict()]
            
    def get_by_id(self, obj_id: str):
        if obj_id in self._store.keys():
            return _create(self._store[obj_id][-1])
        else:
            raise Exception('No instrument with id ' + obj_id + ' exists in storage.')
        
    def _append_values(self, results: dict, keys: list):
        for k,v in self._store.items():
            obj = v[-1]
            for r in results.keys():
                if r == 'num_version':
                    results[r].append(len(v))
                else:
                    results[r].append(obj.get(r))
        
    def get_instrument_list(self, keys=['obj_id', 'cls', 'expiry', 'issue_date' ]):
        num_version = []
        tmp ={k:[] for k in keys}
        tmp['num_version'] = []
        self._append_values(tmp, keys)
        return pd.DataFrame(tmp)

In [5]:
store = InstrumentMemoryStorage()

In [8]:
for days in range (30,90, 30):
    bond = ZeroCouponBondSpecification('BOND_'+str(days), issue_date = dt.datetime(2023,1,1), maturity_date=dt.datetime(2023,1,1) + dt.timedelta(days=days), 
                          currency=Currency.EUR, notional=1.0, issuer='Depp2', securitization_level=SecuritizationLevel.SUBORDINATED)
    store.add(bond)
    
store.add(FixedRateBond('FIXED_BOND_'+str(days), issue_date = dt.datetime(2023,1,1), maturity_date=dt.datetime(2024,1,1), 
                          currency=Currency.EUR, notional=1.0, issuer='Depp', securitisation_level=SecuritizationLevel.SUBORDINATED, 
                        coupon_payment_dates=[dt.datetime(2024,1,1)], coupons=[0.01]))

In [9]:
store.get_instrument_list(keys=['obj_id', 'cls', 'maturity_date', 'issue_date', 'issuer', 'securitisation_level' ])

,obj_id,cls,maturity_date,issue_date,issuer,securitisation_level,num_version
0,BOND_30,ZeroCouponBondSpecification,2023-01-31,2023-01-01,Depp2,None,1
1,BOND_60,ZeroCouponBondSpecification,2023-03-02,2023-01-01,Depp2,None,1
2,FIXED_BOND_60,FixedRateBond,2024-01-01,2023-01-01,Depp,None,1


In [13]:
import rivapy.tools._validators as validators
from rivapy.tools.enums import SecuritizationLevel

validators._enum_to_string(SecuritizationLevel, SecuritizationLevel.COLLATERALIZED.value)


'COLLATERALIZED'